In [1]:
import numpy as np
import pandas as pd
import os
import glob

In [2]:
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
os.chdir('/content/drive/MyDrive/book model datas/practice projects/play_books_data')

In [4]:
df=pd.read_csv('combined_csv.csv')

In [5]:

df.columns

Index(['title', ' author', ' publisher', ' date_published', ' ratings',
       ' pages', ' isbn', ' language', ' genres'],
      dtype='object')

In [6]:
df=df.rename(columns={' title':"title", ' author':'author', ' publisher':'publisher', ' date_published':'date_published', ' ratings':'ratings',
       ' pages':'pages', ' isbn':'isbn', ' language':'language', ' genres':"genre"})

In [7]:
df

,title,author,publisher,date_published,ratings,pages,isbn,language,genre
0,Reading Price Charts Bar by Bar,Al Brooks,John Wiley & Sons,Mar 23| 2009,4.3,432,9780470464274,English,Business & Economics / General
1,Services Marketing,Christopher Lovelock,Pearson Australia,May 20| 2015,4.1,529,9781486004768,English,Business & Economics / Marketing / General
2,Pride & Ownership: A Firefighter's Love of the...,Rick Lasky,PennWell Books,2006,4.5,302,9781593700782,English,Business & Economics / Leadership
3,Social Media Marketing: Edition 3,Tracy L. Tuten,SAGE,Nov 20| 2017,4.7,448,9781526424549,English,Business & Economics / E-Commerce / Internet M...
4,Forex Patterns and Probabilities,Ed Ponsi,John Wiley & Sons,Dec 29| 2010,4.7,272,9781118044926,English,Business & Economics / Foreign Exchange
...,...,...,...,...,...,...,...,...,...
7925,Handbook of Educational Psychology: Edition 3,Lyn Corno,Routledge,Jul 6| 2015,0.0,482,9781317420552,English,Education / Educational Psychology
7926,PSYCH 5| Introductory Psychology| 5th Edition:...,Spencer A. Rathus,Cengage Learning,Dec 5| 2016,5.0,448,9781305888296,English,Education / Teaching Methods & Materials / Gen...
7927,Positive Psychology at Work: How Positive Lead...,Sarah Lewis,John Wiley & Sons,Mar 16| 2011,0.0,272,9781119996217,English,Business & Economics / Leadership
7928,History and Systems of Psychology: Edition 7,James F. Brennan,Cambridge University Press,Oct 26| 2017,0.0,469,9781316834206,English,Education / Teaching Methods & Materials / Gen...


In [8]:
# df.reset_index(drop=True, inplace=True)

In [9]:
df['title'][2464]

'The Goldfinch'

In [10]:
df=df.drop(['publisher', 'date_published',
       'pages', 'isbn', 'language'],axis=1)

In [11]:
df.columns

Index(['title', 'author', 'ratings', 'genre'], dtype='object')

In [12]:
df['author'].nunique()

1192

In [13]:
df['genre']=df['genre'].astype('str')

In [14]:
for i in range(len(df['genre'])):
  df['genre'].loc[i]=df['genre'].loc[i].split('/')

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [15]:
df['genre']

0                       [Business & Economics ,  General]
1          [Business & Economics ,  Marketing ,  General]
2                    [Business & Economics ,  Leadership]
3       [Business & Economics ,  E-Commerce ,  Interne...
4              [Business & Economics ,  Foreign Exchange]
                              ...                        
7925                [Education ,  Educational Psychology]
7926    [Education ,  Teaching Methods & Materials ,  ...
7927                 [Business & Economics ,  Leadership]
7928    [Education ,  Teaching Methods & Materials ,  ...
7929    [Medical ,  Allied Health Services ,  Physical...
Name: genre, Length: 7930, dtype: object

In [16]:
df=df.drop_duplicates(subset=['title'])

In [17]:
df[df['author']=='Al Brooks']

,title,author,ratings,genre
0,Reading Price Charts Bar by Bar,Al Brooks,4.3,"[Business & Economics , General]"
26,Trading Price Action Trends,Al Brooks,4.5,"[Business & Economics , E-Commerce , Online ..."
28,Trading Price Action Reversals,Al Brooks,4.1,"[Business & Economics , General]"
29,Trading Price Action Trading Ranges,Al Brooks,4.8,"[Business & Economics , General]"


In [41]:
# Function for recommending books based on Book title. It takes book title and genre as an input.
def recommend(title, author):
    
    # Matching the genre with the dataset and reset the index
    data = df.loc[df['author'] == author]  
    data.reset_index(level = 0, inplace = True) 
  
    # Convert the index into series
    indices = pd.Series(data.index, index = data['title'])
    
    #Converting the book title into vectors and used bigram
    tf = TfidfVectorizer(analyzer='word', ngram_range=(2, 2), min_df = 1, stop_words='english')
    tfidf_matrix = tf.fit_transform(data['title'])
    
    # Calculating the similarity measures based on Cosine Similarity
    sg = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # Get the index corresponding to original_title
       
    idx = indices[title]
# Get the pairwsie similarity scores 
    sig = list(enumerate(sg[idx]))
# Sort the books
    sig = sorted(sig, key=lambda x: x[1], reverse=True)
# Scores of the 5 most similar books 
    sig = sig[1:6]
    return sig
# Book indicies



In [42]:
recommend(title='Trading Price Action Trading Ranges',author='Al Brooks')

[(1, 0.3605120834049789), (2, 0.3605120834049789), (0, 0.0)]

In [43]:
def get_recommendations(title,author):
  """
  
  """
  # matching genre with the dataset and reset the index
  book_data = df.loc[df['author'] ==author]
  book_data.reset_index(level = 0, inplace = True)

  # getting recommendations
  recoms = recommend(title,author)

  # getiing indexes of book recommendatios
  book_indices = [i[0] for i in recoms]

  # making a list of recommendations
  recommendations = []
  for i in book_indices:
    recommendations.append(book_data['title'].loc[i])
  return recommendations

In [44]:
get_recommendations(title='Trading Price Action Trading Ranges',author='Al Brooks')

['Trading Price Action Trends',
 'Trading Price Action Reversals',
 'Reading Price Charts Bar by Bar']